# Analysis of fitted tracks
Plots many distributions associated with fitted tracks.

## Paths
Set ``datapath`` to point to the directory which contains ``evd`` files.

In [1]:
import ipywidgets as widgets
import numpy as np
import matplotlib.pyplot as plt
import os
import glob
import h5py
import scipy.stats
import matplotlib as mpl
mpl.rc('image', cmap='viridis')

In [2]:
datapath = os.path.join(os.environ['SCRATCH'],'evd/')
# datapath = '/global/project/projectdirs/dune/data/larpix/processed_data/prod_20_10_22/'

## Interactive analysis

In [3]:
def profile_plot(x,y,xbins,method='mean',std_on=False,best_fit=False):
    binned = scipy.stats.binned_statistic(x, [y, y**2], bins=xbins, statistic=method)
    mean, mean2 = binned.statistic
    std = np.sqrt(mean2 - mean**2)
    bin_edges = binned.bin_edges
    bin_centers = (bin_edges[:-1] + bin_edges[1:])/2.

    fmt = 'r.'
    if method != 'mean': fmt = '.'
    plt.errorbar(x=bin_centers, y=mean, yerr=std if std_on else None, fmt=fmt, alpha=0.5)
    
    if best_fit:
        fit = np.polyfit(bin_centers, mean, deg=1, w=std if std_on else None)
        print('Best fit:',fit)
    
def mpv(x):
    val,bins = np.histogram(x,bins=100)
    idx = np.argmax(val)
    return (bins[idx] + bins[idx+1])/2

def vertical_assumption(d):
    rv = dict()
    theta = d['theta']
    phi = d['phi']
    
    rv['start'] = np.where(np.expand_dims(phi>0,axis=-1), d['start'], d['end'])
    rv['end'] = np.where(np.expand_dims(phi>0,axis=-1), d['end'], d['start'])
    rv['theta'] = np.where(phi>0, theta, np.pi - theta)
    rv['phi'] = np.abs(phi)
    rv['residual'] = d['residual']
    return rv

def calc_theta(xyzt):
    return np.arctan2(np.linalg.norm(xyzt[:,:2],axis=-1),xyzt[:,2])

def calc_phi(xyzt):
    return np.arctan2(xyzt[:,1],xyzt[:,0])

def azimuthal(d):
    rv = dict()
    start = d['start'][:]
    end = d['end'][:]
    start[:,[1,2]] = np.array([1,-1]) * d['start'][:,[2,1]] # swap y -> -z, z -> y
    end[:,[1,2]] = np.array([1,-1]) * d['end'][:,[2,1]] # swap y -> -z, z -> y
    theta = calc_theta(end-start)
    phi = calc_phi(end-start)
    residual = d['residual'][:]
    residual[:,[1,2]] = d['residual'][:,[2,1]] # swap y -> -z, z -> y
    
    rv['start'] = start
    rv['end'] = end
    rv['theta'] = theta
    rv['phi'] = phi
    rv['residual'] = residual
    return rv


In [4]:
%matplotlib widget
files = sorted([os.path.basename(path) for path in glob.glob(datapath+'/*.h5')])
@widgets.interact
def display(filename=widgets.Dropdown(options=files, description="File"),
            nhit_cut=widgets.IntText(value=7, description="NHit cut"),
            length_cut=widgets.FloatText(value=0, description="Length cut"),
            residual_cut=widgets.FloatText(value=300, description="Residual cut"),
            theta_cut=widgets.FloatText(value=0, description="Theta cut"),
            event_frac_cut=widgets.FloatText(value=0, description="Event fraction cut"),
            assume_vertical=widgets.ToggleButton(value=True, description="Assume vertical tracks"),
            azimuthal_coords=widgets.ToggleButton(value=False, description="Azimuthal coordinates")
           ):
    plt.close('all')
    f = h5py.File(os.path.join(datapath,filename),'r')
    if len(f['tracks']):            
        event_hits = np.array([f['events'][ f['tracks'][i]['event_ref'] ]['nhit'][0] for i in range(len(f['tracks']))])
        event_fraction = f['tracks']['nhit']/event_hits
        good_track_mask = np.ones(len(f['tracks'])).astype(bool)
        good_track_mask = np.logical_and(f['tracks']['nhit'] > nhit_cut, good_track_mask)
        good_track_mask = np.logical_and(f['tracks']['length'] > length_cut, good_track_mask)
        good_track_mask = np.logical_and(np.linalg.norm(f['tracks']['residual'][:,:3],axis=-1) < residual_cut, good_track_mask)
        good_track_mask = np.logical_and(np.abs(f['tracks']['theta']) > theta_cut, good_track_mask)
        good_track_mask = np.logical_and(np.abs(np.pi - f['tracks']['theta']) > theta_cut, good_track_mask)
        good_track_mask = np.logical_and(event_fraction > event_frac_cut, good_track_mask)
        
        if assume_vertical:
            v = vertical_assumption(f['tracks'])
            theta = v['theta']
            phi = v['phi']
            start = v['start']
            end = v['end']
            residual = v['residual']
        else:
            theta = f['tracks']['theta']
            phi = f['tracks']['phi']
            start = f['tracks']['start']
            end = f['tracks']['end']
            residual = f['tracks']['residual']
        if azimuthal_coords:
            a = azimuthal(f['tracks'] if not assume_vertical else v)
            theta = a['theta']
            phi = a['phi']
            start = a['start']
            end = a['end']
            residual = a['residual']
            
        plt.figure('track nhit')
        plt.hist(f['tracks']['nhit'],bins=np.linspace(0,100,100),histtype='step')
        plt.hist(f['tracks']['nhit'][good_track_mask],bins=np.linspace(0,100,100),histtype='step')
        plt.xlabel('nhit')
        plt.legend(('all tracks','selected tracks'))
        
        plt.figure('track nhit fraction')
        plt.hist(event_fraction,bins=np.linspace(0,1,100),histtype='step')
        plt.hist(event_fraction[good_track_mask],bins=np.linspace(0,1,100),histtype='step')
        plt.xlabel('fraction of event hits')
        plt.legend(('all tracks','selected tracks'))
        
        plt.figure('track length')
        plt.hist(f['tracks']['length'],bins=np.linspace(0,440,100),histtype='step')
        plt.hist(f['tracks']['length'][good_track_mask],bins=np.linspace(0,440,100),histtype='step')
        plt.xlabel('track length [mm]')
        plt.legend(('all tracks','selected tracks'))
        
        plt.figure('track theta')
        plt.hist(theta,bins=np.linspace(0,np.pi,100),histtype='step')
        plt.hist(theta[good_track_mask],bins=np.linspace(0,np.pi,100),histtype='step')
        plt.xlabel('track theta [rad.]')
        plt.legend(('all tracks','selected tracks'))
        plt.axvline(np.pi/2)
        
        plt.figure('track phi')
        plt.hist(phi,bins=np.linspace(-np.pi,np.pi,100),histtype='step')
        plt.hist(phi[good_track_mask],bins=np.linspace(-np.pi,np.pi,100),histtype='step')
        plt.xlabel('track phi [rad.]')
        plt.legend(('all tracks','selected tracks'))
        
        plt.figure('track res x')
        plt.hist(residual[:,0],bins=np.linspace(0,10,100),histtype='step')
        plt.hist(residual[good_track_mask,0],bins=np.linspace(0,10,100),histtype='step')
        plt.xlabel('track residual x [mm]')
        plt.legend(('all tracks','selected tracks'))
        
        plt.figure('track res y')
        plt.hist(residual[:,1],bins=np.linspace(0,10,100),histtype='step')
        plt.hist(residual[good_track_mask,1],bins=np.linspace(0,10,100),histtype='step')
        plt.xlabel('track residual y [mm]')
        plt.legend(('all tracks','selected tracks'))
        
        plt.figure('track res z')
        plt.hist(residual[:,2],bins=np.linspace(0,10,100),histtype='step')
        plt.hist(residual[good_track_mask,2],bins=np.linspace(0,10,100),histtype='step')
        plt.xlabel('track residual z [mm]')
        plt.legend(('all tracks','selected tracks'))
        
        plt.figure('track res')
        plt.hist(np.linalg.norm(residual[:,:3],axis=-1),bins=np.linspace(0,10,100),histtype='step')
        plt.hist(np.linalg.norm(residual[good_track_mask,:3],axis=-1),bins=np.linspace(0,10,100),histtype='step')
        plt.xlabel('track residual [mm]')
        plt.legend(('all tracks','selected tracks'))
        
        plt.figure('track q')
        plt.hist(f['tracks']['q']*0.250,bins=np.linspace(0,1e3,100),histtype='step')
        plt.hist(f['tracks']['q'][good_track_mask]*0.250,bins=np.linspace(0,1e3,100),histtype='step')
        plt.xlabel('track q [ke]')
        plt.legend(('all tracks','selected tracks'))
        
        plt.figure('track time length')
        plt.hist(f['tracks']['ts_end'] - f['tracks']['ts_start'],bins=np.linspace(0,400,100),histtype='step')
        plt.hist((f['tracks']['ts_end'] - f['tracks']['ts_start'])[good_track_mask],bins=np.linspace(0,400,100),histtype='step')
        plt.xlabel('track dt [0.1us]')
        plt.legend(('all tracks','selected tracks'))
        
        plt.figure('track depth')
        plt.hist(np.abs(end[:,2] - start[:,2]),bins=np.linspace(0,50,100),histtype='step')
        plt.hist(np.abs(end[:,2] - start[:,2])[good_track_mask],bins=np.linspace(0,50,100),histtype='step')
        plt.xlabel('track depth [mm]')
        plt.legend(('all tracks','selected tracks'))
        
        plt.figure('track dQ/dx')
        dqdx = f['tracks']['q']*0.250/f['tracks']['length']
        plt.hist(dqdx,bins=np.linspace(0,10,100),histtype='step')
        plt.hist(dqdx[good_track_mask],bins=np.linspace(0,10,100),histtype='step')
        plt.xlabel('track mean dQ/dx [ke/mm]')
        plt.legend(('all tracks','selected tracks'))

#         good_track_mask = np.ones(len(f['tracks'])).astype(bool)
        
        x_start = start[good_track_mask,0]
        y_start = start[good_track_mask,1]
        z_start = start[good_track_mask,2]
        x_end = end[good_track_mask,0]
        y_end = end[good_track_mask,1]
        z_end = end[good_track_mask,2]
        plt.figure('track end points (x,y)')
        fig, axes = plt.subplots(1, 2, num='track end points (x,y)', sharex=True, sharey=True)
        axes[0].hist2d(x_start,y_start,bins=(np.linspace(-166.275,166.275,75),np.linspace(-166.275,166.275,75)),cmin=0.5)
        plt.colorbar(axes[1].hist2d(x_end,y_end,bins=(np.linspace(-166.275,166.275,75),np.linspace(-166.275,166.275,75)),cmin=0.5)[-1])
        plt.xlabel('x [mm]')
        axes[0].set_ylabel('y [mm]')
        axes[0].set_title('track start')
        axes[1].set_title('track end')
        fig, axes = plt.subplots(1, 2, num='track end points (x,z)', sharex=True, sharey=True)
        axes[0].hist2d(x_start,z_start,bins=(np.linspace(-166.275,166.275,75),np.linspace(-166.275,166.275,75)),cmin=0.5)
        plt.colorbar(axes[1].hist2d(x_end,z_end,bins=(np.linspace(-166.275,166.275,75),np.linspace(-166.275,166.275,75)),cmin=0.5)[-1])
        plt.xlabel('x [mm]')
        axes[0].set_ylabel('z [mm]')
        axes[0].set_title('track start')
        axes[1].set_title('track end')
        fig, axes = plt.subplots(1, 2, num='track end points (y,z)', sharex=True, sharey=True)
        axes[0].hist2d(z_start,y_start,bins=(np.linspace(-166.275,166.275,75),np.linspace(-166.275,166.275,75)),cmin=0.5)
        plt.colorbar(axes[1].hist2d(z_end,y_end,bins=(np.linspace(-166.275,166.275,75),np.linspace(-166.275,166.275,75)),cmin=0.5)[-1])
        plt.xlabel('z [mm]')
        axes[0].set_ylabel('y [mm]')
        axes[0].set_title('track start')
        axes[1].set_title('track end')
        
        plt.figure('track dQ/dx v. theta')
        plt.hist2d(theta[good_track_mask],dqdx[good_track_mask],bins=(np.linspace(0,np.pi,100),np.linspace(0,10,100)))
        plt.colorbar()
        profile_plot(theta[good_track_mask],dqdx[good_track_mask],np.linspace(0,np.pi,100))
        profile_plot(theta[good_track_mask],dqdx[good_track_mask],np.linspace(0,np.pi,100), method='median')
        profile_plot(theta[good_track_mask],dqdx[good_track_mask],np.linspace(0,np.pi,100), method=mpv)
        plt.legend(['mean','median','mpv'])
        plt.ylabel('track mean dQ/dx [ke/mm]')
        plt.xlabel('track theta [rad.]')
        plt.axvline(np.pi/2)
        
        plt.figure('track dQ/dx v. phi')
        plt.hist2d(phi[good_track_mask],dqdx[good_track_mask],bins=(np.linspace(-np.pi,np.pi,100),np.linspace(0,10,100)))
        plt.colorbar()
        profile_plot(phi[good_track_mask],dqdx[good_track_mask],np.linspace(-np.pi,np.pi,100))
        profile_plot(phi[good_track_mask],dqdx[good_track_mask],np.linspace(-np.pi,np.pi,100), method='median')
        profile_plot(phi[good_track_mask],dqdx[good_track_mask],np.linspace(-np.pi,np.pi,100), method=mpv)
        plt.legend(['mean','median','mpv'])
        plt.ylabel('track mean dQ/dx [ke/mm]')
        plt.xlabel('track phi [rad.]')
        
        plt.figure('track dQ/dx v. residual')
        plt.hist2d(np.linalg.norm(residual[good_track_mask,:3],axis=-1),dqdx[good_track_mask],bins=(np.linspace(0,10,100),np.linspace(0,10,100)))
        plt.colorbar()
        profile_plot(np.linalg.norm(residual[good_track_mask,:3],axis=-1),dqdx[good_track_mask],np.linspace(0,10,100), method='mean')
        profile_plot(np.linalg.norm(residual[good_track_mask,:3],axis=-1),dqdx[good_track_mask],np.linspace(0,10,100), method='median')
        profile_plot(np.linalg.norm(residual[good_track_mask,:3],axis=-1),dqdx[good_track_mask],np.linspace(0,10,100), method=mpv)
        plt.legend(['mean','median','mpv'])
        plt.ylabel('track mean dQ/dx [ke/mm]')
        plt.xlabel('track residual [mm]')
        
        plt.figure('track dQ/dx v. length')
        plt.hist2d(f['tracks']['length'][good_track_mask],dqdx[good_track_mask],bins=(np.linspace(0,440,100),np.linspace(0,10,100)))
        plt.colorbar()
        profile_plot(f['tracks']['length'][good_track_mask],dqdx[good_track_mask],np.linspace(0,440,100), method='mean')
        profile_plot(f['tracks']['length'][good_track_mask],dqdx[good_track_mask],np.linspace(0,440,100), method='median')
        profile_plot(f['tracks']['length'][good_track_mask],dqdx[good_track_mask],np.linspace(0,440,100), method=mpv)
        plt.legend(['mean','median','mpv'])
        plt.ylabel('track mean dQ/dx [ke/mm]')
        plt.xlabel('track length [mm]')
        
        plt.figure('track dQ/dx v. Q')
        plt.hist2d(f['tracks']['q'][good_track_mask],dqdx[good_track_mask],bins=(np.linspace(0,1e3,100),np.linspace(0,10,100)))
        plt.colorbar()
        profile_plot(f['tracks']['q'][good_track_mask],dqdx[good_track_mask],np.linspace(0,1e3,100), method='mean')
        profile_plot(f['tracks']['q'][good_track_mask],dqdx[good_track_mask],np.linspace(0,1e3,100), method='median')
        profile_plot(f['tracks']['q'][good_track_mask],dqdx[good_track_mask],np.linspace(0,1e3,100), method=mpv)
        plt.legend(['mean','median','mpv'])
        plt.ylabel('track mean dQ/dx [ke/mm]')
        plt.xlabel('track Q [ke]')
        
        plt.figure('track residual v. theta')
        plt.hist2d(theta[good_track_mask],np.linalg.norm(residual[good_track_mask,:3],axis=-1),bins=(np.linspace(0,np.pi,100),np.linspace(0,10,100)))
        plt.colorbar()
        profile_plot(theta[good_track_mask],np.linalg.norm(residual[good_track_mask,:3],axis=-1),np.linspace(0,np.pi,100))
        plt.ylabel('track residual [mm]')
        plt.xlabel('track theta [rad.]')
        plt.axvline(np.pi/2)
        
        plt.figure('track length v. theta')
        plt.hist2d(theta[good_track_mask],f['tracks']['length'][good_track_mask],bins=(np.linspace(0,np.pi,100),np.linspace(0,440,100)))
        plt.colorbar()
        profile_plot(theta[good_track_mask],f['tracks']['length'][good_track_mask],np.linspace(0,np.pi,100))
        plt.ylabel('track length [mm]')
        plt.xlabel('track theta [rad.]')
        plt.axvline(np.pi/2)
        
        plt.figure('track length v. residual')
        plt.hist2d(f['tracks']['length'][good_track_mask],np.linalg.norm(residual[good_track_mask,:3],axis=-1),bins=(np.linspace(0,440,100),np.linspace(0,10,100)))
        plt.colorbar()
        profile_plot(f['tracks']['length'][good_track_mask],np.linalg.norm(residual[good_track_mask,:3],axis=-1),np.linspace(0,440,100))
        plt.xlabel('track length [mm]')
        plt.ylabel('track residual [mm]')
        
        plt.figure('track charge v. nhit')
        plt.hist2d(f['tracks']['nhit'][good_track_mask],f['tracks']['q'][good_track_mask]*0.250,bins=(np.linspace(0,100,100),np.linspace(0,1e3,100)))
        plt.colorbar()
        profile_plot(f['tracks']['nhit'][good_track_mask],f['tracks']['q'][good_track_mask]*0.250,np.linspace(0,100,100))
        plt.xlabel('track n hits')
        plt.ylabel('track q [ke]')
        
        plt.figure('track dQ/dx v. mean hit q')
        q_mean = f['tracks']['q']*0.250/f['tracks']['nhit']
        plt.hist2d(q_mean[good_track_mask],dqdx[good_track_mask],bins=(np.linspace(0,30,100),np.linspace(0,10,100)))
        plt.colorbar()
        profile_plot(q_mean[good_track_mask],dqdx[good_track_mask],np.linspace(0,30,100))
        profile_plot(q_mean[good_track_mask],dqdx[good_track_mask],np.linspace(0,30,100), method='median')
        profile_plot(q_mean[good_track_mask],dqdx[good_track_mask],np.linspace(0,30,100), method=mpv)
        plt.legend(['mean','median','mpv'])
        plt.xlabel('track mean hit q [ke]')
        plt.ylabel('track dQ/dx [ke/mm]')
        
        plt.figure('track dQ/dx v. nhit')
        plt.hist2d(f['tracks']['nhit'][good_track_mask],dqdx[good_track_mask],bins=(np.linspace(0,100,100),np.linspace(0,10,100)))
        plt.colorbar()
        profile_plot(f['tracks']['nhit'][good_track_mask],dqdx[good_track_mask],np.linspace(0,100,100))
        profile_plot(f['tracks']['nhit'][good_track_mask],dqdx[good_track_mask],np.linspace(0,100,100), method='median')
        profile_plot(f['tracks']['nhit'][good_track_mask],dqdx[good_track_mask],np.linspace(0,100,100), method=mpv, best_fit=True)
        plt.legend(['mean','median','mpv'])
        plt.xlabel('track n hits')
        plt.ylabel('track dQ/dx [ke/mm]')
    else:
        print('no events in file')

interactive(children=(Dropdown(description='File', options=('datalog_2020_10_12_16_24_35_PDT_evd.h5', 'datalog…